<a href="https://colab.research.google.com/github/bnnguyen/DESLab_ML_training_2024/blob/main/Deslab_2024_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#KNN - project mẫu

Trước hết, hãy hiểu sơ qua về thuật toán kNN tại [đây](https://www.youtube.com/watch?v=0p0o5cmgLdE)

Trong project này, chúng ta sẽ làm việc với bộ dữ liệu Iris, một bộ dữ liệu có sẵn trong thư viện sklearn.

Bộ dữ liệu Iris chứa bốn đặc điểm (chiều dài và chiều rộng của lá đài và cánh hoa) của 50 mẫu của ba loài Iris (Iris setosa, Iris virginica và Iris versicolor). Và ở đây, chúng ta sẽ tạo ra một mô hình KNN phân để phân loại (dán nhãn/phân nhãn) các loài.

In [ ]:
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# Load the Iris dataset
iris = datasets.load_iris()

# Create a DataFrame from the Iris data
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# Add the target column to the DataFrame
iris_df['target'] = iris.target

# Create a dictionary to map the target values to class names
target_names = {0: 'setosa', 1: 'versicolor', 2: 'virginica'}

# Use the map function to replace the target values with class names
iris_df['target'] = iris_df['target'].map(target_names)

columns_to_display = iris_df.columns[:5]

# Use the pd.concat() function to concatenate the first 5 columns of the DataFrame
display(pd.concat([iris_df[col] for col in columns_to_display], axis=1))

Chà, những bài trước và bài này đều là rất nhiều code đấy. Phần code bên trên cũng có hơi "đồ sộ", nhưng đừng nản lòng nhé! Việc chúng ta đang tập trung làm bây giờ là hiển thị dữ liệu ở dạng có thể đọc được để chúng ta có thể thao tác và làm việc trên nó.

Trong biểu diễn DataFrame ở trên, bạn có thể thấy rằng 4 cột đầu tiên là số liệu đo lường các đặc điểm và cột cuối cùng là phân loại (nhãn) của cá thể mang các đặc điểm đó!

...nhưng, vẫn còn nhiều số quá ta! Hay biểu diễn bộ dữ liệu bằng biểu đồ có sự phân biệt giữa các loài hoa xem sao?

In [ ]:
def plot_iris_features(data):
    # Get the feature and label arrays

    X = data.data
    y = data.target
    feature_names = data.feature_names
    # Create a list of colors for each class
    colors = ['red', 'green', 'blue']
    # Create a list of the indices of each class
    class_indices = [np.where(y == i) for i in range(3)]
    # Create a list of all possible combinations of 2 features
    feature_combinations = [(i, j) for i in range(4) for j in range(i+1, 4)]
    # Create a subplot for each combination of 2 features
    for i, (x_index, y_index) in enumerate(feature_combinations):
        plt.subplot(2, 3, i+1)
        for class_index, color in zip(class_indices, colors):
            plt.scatter(X[class_index, x_index], X[class_index, y_index], c=color)
        plt.xlabel(feature_names[x_index])
        plt.ylabel(feature_names[y_index])
        plt.tight_layout()

    return plt

Hàm này sẽ giúp chúng ta vẽ một số biểu đồ và cho chúng ta thấy mối quan hệ giữa các đặc điểm của dữ liệu và nhãn của chúng.

In [ ]:
all_plots = plot_iris_features(iris)
all_plots

Biểu đồ của chúng ta cho thấy điều gì? Mỗi biểu đồ trong số 6 biểu đồ chọn cùng lúc 2 đặc điểm và sau đó biểu diễn đo lường 2 đặc điểm đó của tất cả 50 mẫu hoa trên biểu đồ 2D. Chúng ta đi trước một bước và tô màu điểm của mỗi loài hoa bằng những màu khác nhau.

Bạn đã có thể nhìn thấy lợi ích của việc vẽ ra biểu đồ. Thông qua kỹ thuật này, chúng ta đã có hình dung chung về một số đặc điểm:



* Dữ liệu này có thể phân tách rõ ràng (mỗi loài hoa có một giới hạn chiều dài cũng như là chiều rộng cánh hoa, đài hoa riêng). Điều này có nghĩa là vấn đề phân loại với bộ dữ liệu này không phải là không thể giải quyết được.
* Một số tổ hợp các đặc điểm (như chiều dài cánh hoa và chiều dài đài hoa) là sự kết hợp tốt để phân biệt các loài hoa khác nhau (vì các đặc điểm này có sự chênh lệch rất rõ ràng/lớn giữa những loài hoa khác nhau). Những sự kết hợp khác khác thì không...

Bây giờ hãy tiếp tục và triển khai KNN.

Đầu tiên chúng ta bắt đầu với một class. Class là đơn vị logic của chương trình Python (hoặc bất kỳ ngôn ngữ nào). Một class xác định cấu trúc cho đơn vị logic này. Mỗi sự khởi tạo riêng lẻ của class được gọi là một unit.

Một unit của class thường chứa một số giá trị dữ liệu cụ thể cho unit đó. Tất cả các biến `self.something` đều có nghĩa là chúng là thuộc tính dữ liệu của riêng một class.

Một class cũng định nghĩa một số hàm có thể nhận một số tham số bên ngoài và áp dụng tính toán trên những hàm sử dụng một số dữ liệu mà chúng có.

Trước tiên, chúng ta xác định một class `KNeighborsClassifierCustom` sẽ là class KNN. Việc này chỉ tính đến một tham số - giá trị của `k`.

In [ ]:
import numpy as np

class KNeighborsClassifierCustom:
    def __init__(self, n_neighbors):
        self.n_neighbors = n_neighbors

    # X is a list of multiple points!
    # X could look like
    # [[11, 22, 14],
    #. [53, 36, 57],
    #. [34, 22, 19]]
    # For every point in X you will have a label. y could look like ['A', 'A', 'B']
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    # X is a list of multiple points!
    # X could look like
    # [[1, 2, 4],
    #. [3, 6, 7],
    #. [4, 2, 9]]
    # This means you're asked to find the label of points (1, 2, 4), (3, 6, 7), (4, 2, 9)
    # If there are 2 labels A and B, your example return y_pred_all could look like ['A', 'B', 'B']
    def predict(self, X):
        y_pred_all = []
        for x_test in X:
          distances = np.sqrt(np.sum((self.X_train - x_test)**2, axis=1))
          nearest_indices = np.argsort(distances, axis=0)[:self.n_neighbors]
          nearest_labels = self.y_train[nearest_indices]
          (values,counts) = np.unique(nearest_labels, return_counts=True)
          ind=np.argmax(counts)
          y_pred_all.append(values[ind])
        return y_pred_all

    def score(self, X, y):
        y_pred = self.predict(X)
        return np.mean(y_pred == y)




Tiếp theo chúng ta thiết kế một hàm phù hợp. Thông thường, tất cả các class dành cho thuật toán Machine Learning đều thực hiện trích xuất mẫu và phân tích dữ liệu tại hàm này. Nhưng hãy nhớ rằng trong KNN, chúng ta không trích xuất bất kỳ mẫu nào! Hay nói đúng hơn, chúng ta chỉ dùng các điểm được nhập, không cần qua xử lý toán học, và sử dụng chúng để tìm những điểm lân cận gần nhất của bất kỳ điểm nào mà chúng ta được yêu cầu phân loại. Do đó, hàm `fit` của chúng ta chỉ lưu trữ dữ liệu huấn luyện được cung cấp cho nó.

Bây giờ là lúc thiết kế hàm `predict` của chúng ta. Đối với mỗi điểm trong X, chúng ta tính toán khoảng cách của nó với tất cả các điểm trong `self.X_train`, sau đó chọn `n_neighbors` trên cùng và kiểm tra xem nhãn của chúng là gì. Sau đó, chúng ta tính toán nhãn nào xuất hiện thường xuyên nhất và thêm nó vào danh sách dự đoán của chúng ta. Cuối cùng chúng ta trả lại tất cả các dự đoán.

Phần tiếp theo là thiết kế hàm `score`. Với một số điểm dữ liệu (`X`) và nhãn của chúng (`y`), chúng ta có thể sử dụng thuật toán dự đoán của riêng mình và kiểm tra xem nó chính xác đến mức nào. Vì vậy, chúng ta chuyển tất cả các điểm cho hàm `predict`, lấy các nhãn được dự đoán và sau đó phép so sánh `(y_pred == y)` cung cấp cho chúng ta giá trị 0 hoặc 1. (0 cho mỗi điểm chúng ta sai, 1 cho mỗi điểm chúng ta đúng.) Chúng ta lấy giá trị trung bình và gọi đó là độ hiệu quả của mô hình.

# Kiểm tra độ hiệu quả của mô hình được tạo ra với KNN

Chúng ta cũng muốn so sánh xem giá trị nào của `k` (số cụm phân loại nhãn) mang lại cho chúng ta số lượng mẫu đúng tối đa! Cách tốt nhất để làm điều này là quan sát nó - và đó chính xác là những gì chúng ta sẽ làm.

Khi bạn có được phần trăm chính xác cho các giá trị khác nhau của `k`, việc vẽ thông tin đó dưới dạng biểu đồ cột là khá đơn giản. Bạn chỉ cần xác định những giá trị nào sẽ nằm trên trục X và các giá trị tương ứng trên trục Y.

Matplotlib cung cấp cho chúng ta một số hàm hữu ích để vẽ biểu đồ cột mà chúng ta đang cần.

In [ ]:
def plot_accuracy(accuracy_per_k, classifier_name):
    # Extract the values of k and accuracy from the dictionary
    k_values = list(accuracy_per_k.keys())
    accuracy = list(accuracy_per_k.values())
    # Create a bar chart of the accuracy per k
    plt.bar(k_values, accuracy)
    plt.xlabel("k")
    plt.ylabel("Accuracy (%)")
    plt.title(f"Accuracy of KNN per k for {classifier_name}")
    plt.show()

Bây giờ chúng ta đã có tất cả các hàm cần thiết - công việc của chúng ta gần như đã hoàn thành. Tất cả những gì cần làm là một hàm có thể điền các giá trị khác nhau của `k` đồng thời tính toán độ chính xác trên từng giá trị của `k`, rồi cuối cùng vẽ biểu đồ cột bằng cách sử dụng các giá trị đó.

In [ ]:
def find_best_k(X, y):
    best_k = 0
    best_accuracy = 0
    accuracy_per_k = {}
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
    for k in range(1, 11):
        classifier = KNeighborsClassifierCustom(k)
        classifier.fit(X_train, y_train)
        accuracy_per_k[k] = classifier.score(X_test, y_test)
        if accuracy_per_k[k] > best_accuracy:
            best_k = k
            best_accuracy = accuracy_per_k[k]
    # call the plot_accuracy function
    plot_accuracy(accuracy_per_k, "Classifier")
    return best_k, best_accuracy

Cùng xem kết quả nhé!

In [ ]:
iris = datasets.load_iris()
best_k, best_score = find_best_k(iris.data, iris.target)
print(f"The best value of k is {best_k} with accuracy {best_score * 100}%")

# Scikit-Learn

Biết cách các thuật toán được triển khai là điều tuyệt vời vì nó sẽ giúp bạn giải thích lý do tại sao một số thuật toán hoạt động trên dữ liệu của bạn.

Bây giờ, bạn đã biết chính xác thuật toán *KNN* hoạt động như thế nào. Tuy nhiên, chúng ta hiếm khi tự triển khai thuật toán này nói riêng, cũng như các thuật toán Machine Learning nói chung từ đầu lắm ấy 🤔, bởi lẽ chúng ta có sự hỗ trợ từ các hàm có sẵn trong thư viện Scikit-Learn.

Scikit-learn là một trong những thư viện máy học đang phổ biến nhất trong ngành. Với sự hỗ trợ của nó, bạn sẽ không có lỗi không mong muốn và có hiệu suất cực kỳ cao.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

def find_best_k_per_classifier(X, y):
    best_k = {}
    best_accuracy = {}
    accuracy_per_k_per_classifier = {"sklearn": {}}  # Only include sklearn for now

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    for k in range(1, 11):
        sklearn_classifier = KNeighborsClassifier(k)
        sklearn_classifier.fit(X_train, y_train)
        accuracy_per_k_per_classifier["sklearn"][k] = sklearn_classifier.score(X_test, y_test)
    plot_accuracy(accuracy_per_k_per_classifier["sklearn"], "SklearnKNN")

    return [max(accuracy_per_k_per_classifier.get(key, {}).items(), key=lambda x: x[1]) for key in accuracy_per_k_per_classifier.keys()]

iris = datasets.load_iris()
accs_and_ks = find_best_k_per_classifier(iris.data, iris.target)

print(f"The best accuracy for Sklearn KNN is at k = {accs_and_ks[0][0]} with accuracy {accs_and_ks[0][1] * 100}%")
